In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [162]:
def get_city_dummies(data, n_dummies):
    city_dummies = pd.get_dummies(data['city'])
    index = data['city'].value_counts().head(n_dummies).index
    return pd.concat([data, city_dummies[index]], axis=1)

#### Read all data to dataframe

In [17]:
path = '/Users/ivylai/Desktop/Galvanize/clean_apt_data.csv'
data = pd.read_csv(path)
data.drop(columns = ['Unnamed: 0','url','property_name'], inplace = True)
data = get_city_dummies(data, 3)
data.rename(columns = {'rent_updated':'Rental Price','n_bed':'Number of Beds','n_bath':'Number of Baths',
                        'sq_ft_updated':'SQ-FT','allow_pet':'Allow Pets','minimum_lease':'Minimum Lease Length',
                        'onsite_parking':'Onsite Parking','pool':'Pool','fitness_center':'Fitness Center', 
                        'elevator':'Elevator', 'kitchen_features': 'Number of Kitchen Features', 
                        'security_system':'Security System','washer_dryer':'Washer/Dryer', 'internet':'WiFi', 
                        'air_conditioning':'Air Conditioning', 'furnished':'Furnished','walkscore':'Walkscore'},inplace=True)
data.drop(columns = 'city', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15588 entries, 0 to 15587
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rental Price                15588 non-null  float64
 1   Number of Beds              15588 non-null  int64  
 2   Number of Baths             15588 non-null  float64
 3   SQ-FT                       15588 non-null  float64
 4   Walkscore                   15588 non-null  float64
 5   Allow Pets                  15588 non-null  int64  
 6   Minimum Lease Length        15588 non-null  int64  
 7   Onsite Parking              15588 non-null  int64  
 8   Pool                        15588 non-null  int64  
 9   Fitness Center              15588 non-null  int64  
 10  Elevator                    15588 non-null  int64  
 11  Number of Kitchen Features  15588 non-null  int64  
 12  Security System             15588 non-null  int64  
 13  Washer/Dryer                155

#### Read data with no range to dataframe

In [18]:
path = '/Users/ivylai/Desktop/Galvanize/clean_apt_data_norange.csv'
data_norange = pd.read_csv(path)
data_norange.drop(columns = ['Unnamed: 0','url','property_name','rent_range','sq_ft_range'], inplace = True)
data_norange = get_city_dummies(data_norange, 3)
data_norange.rename(columns = {'rent_updated':'Rental Price','n_bed':'Number of Beds','n_bath':'Number of Baths',
                        'sq_ft_updated':'SQ-FT','allow_pet':'Allow Pets','minimum_lease':'Minimum Lease Length',
                        'onsite_parking':'Onsite Parking','pool':'Pool','fitness_center':'Fitness Center', 
                        'elevator':'Elevator', 'kitchen_features': 'Number of Kitchen Features', 
                        'security_system':'Security System','washer_dryer':'Washer/Dryer', 'internet':'WiFi', 
                        'air_conditioning':'Air Conditioning', 'furnished':'Furnished','walkscore':'Walkscore'},inplace=True)
data_norange.drop(columns = 'city', inplace=True)
data_norange.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7122 entries, 0 to 7121
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rental Price                7122 non-null   float64
 1   Number of Beds              7122 non-null   int64  
 2   Number of Baths             7122 non-null   float64
 3   SQ-FT                       7122 non-null   float64
 4   Walkscore                   7122 non-null   float64
 5   Allow Pets                  7122 non-null   int64  
 6   Minimum Lease Length        7122 non-null   int64  
 7   Onsite Parking              7122 non-null   int64  
 8   Pool                        7122 non-null   int64  
 9   Fitness Center              7122 non-null   int64  
 10  Elevator                    7122 non-null   int64  
 11  Number of Kitchen Features  7122 non-null   int64  
 12  Security System             7122 non-null   int64  
 13  Washer/Dryer                7122 

## Modeling

In [55]:
def cross_val(X_train, y_train, k, regression = None):
    kf = KFold(n_splits=k, shuffle=True)
    scores = []
    
    if regression != None:
        scaler = StandardScaler()
        X_std = scaler.fit_transform(X_train)
        model = regression
        for train, test in kf.split(X_std):
            model.fit(X_std[train], y_train.iloc[train])
            y_pred = model.predict(X_std[test])
            scores.append(mean_squared_error(y_train.iloc[test], y_pred))
            
    else:
        for train, test in kf.split(X_train):
            y_pred = np.full(y_train.iloc[test].shape, np.mean(y_train.iloc[train]))
            scores.append(mean_squared_error(y_train.iloc[test], y_pred))
    
    average_score = round(np.sqrt(np.mean(scores)),2)
        
    return average_score

In [94]:
#Baseline
def baseline_train_validate(X_train, y_train, k):
    baseline_train = cross_val(X_train, y_train, k, regression = None)
    print(f"Baseline RMSE: {baseline_train}")

In [95]:
# Linear Regression
def lin_reg_train_validate(X_train, y_train, k):
    lin_reg = LinearRegression()
    lin_reg_train = cross_val(X_train, y_train, k, regression = lin_reg)
    print(f"Linear Regression RMSE: {lin_reg_train}")
    return lin_reg

In [96]:
# Ridge Regression
def ridge_ref_train_validate(alpha_lower, alpha_upper, X_train, y_train, k):
    alphas = np.linspace(alpha_lower, alpha_upper, 1000)
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_train)
    regr_cv = RidgeCV(alphas=alphas)
    model_cv = regr_cv.fit(X_std, y_train)
    best_alpha = round(model_cv.alpha_,1)
    print(f"Best alpha is: {best_alpha}")
    ridge_reg = Ridge(alpha = best_alpha)
    ridge_reg_train = cross_val(X_train, y_train, k, regression = ridge_reg)
    print(f"Ridge Regression RMSE: {ridge_reg_train}")
    return ridge_reg

In [103]:
# Decision Tree
def decision_tree_train_validaion(dt_grid, X_train, y_train, k):
    dt_gridsearch = GridSearchCV(DecisionTreeRegressor(),
                                 dt_grid,
                                 n_jobs=-1,
                                 verbose=False,
                                 scoring='neg_mean_squared_error')
    dt_gridsearch.fit(X_train, y_train)
    print("Best parameters:", dt_gridsearch.best_params_)
    dt_model = dt_gridsearch.best_estimator_
    dt_train = cross_val(X_train, y_train, k, regression = dt_model)
    print(f"Decision Tree RMSE: {dt_train}")
    return dt_model

In [110]:
# Random Forest
def random_forest_train_validaion(rf_grid, X_train, y_train, k):
    rf_gridsearch = GridSearchCV(RandomForestRegressor(),
                                 rf_grid,
                                 n_jobs=-1,
                                 verbose=False,
                                 scoring='neg_mean_squared_error')
    rf_gridsearch.fit(X_train, y_train)
    print("Best parameters:", rf_gridsearch.best_params_)
    rf_model = rf_gridsearch.best_estimator_
    rf_train = cross_val(X_train, y_train, k, regression = rf_model)
    print(f"Random Forest RMSE: {rf_train}")
    return rf_model

In [112]:
# Gradient Boosting
def gradient_boosting_train_validaion(gb_grid, n_estimators, X_train, y_train, k):
    gb_gridsearch = GridSearchCV(GradientBoostingRegressor(n_estimators=n_estimators),
                                 gb_grid,
                                 n_jobs=-1,
                                 verbose=False,
                                 scoring='neg_mean_squared_error')
    gb_gridsearch.fit(X_train, y_train)
    print("Best parameters:", gb_gridsearch.best_params_)
    gb_model = gb_gridsearch.best_estimator_
    gb_train = cross_val(X_train, y_train, k, regression = gb_model)
    print(f"Random Forest RMSE: {gb_train}")
    return gb_model

In [117]:
def final_testing(X_train, X_test, y_train, y_test, model):
    model_name = type(model).__name__
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = round(np.sqrt(mean_squared_error(y_test,y_pred)),2)
    print(f'Final RMSE for {model_name}: {score}')
    return score

#### Train and Test on All Data

In [47]:
X_all = data.drop(columns = 'Rental Price')
y_all = data['Rental Price']
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(X_all, y_all)

In [123]:
baseline_train_validate(X_all_train, y_all_train, 10)
lin_reg = lin_reg_train_validate(X_all_train, y_all_train, 10)
print("")
ridge_reg = ridge_ref_train_validate(50, 250, X_all_train, y_all_train, 10)

Baseline RMSE: 1871.82
Linear Regression RMSE: 1428.83

Best alpha is: 83.4
Ridge Regression RMSE: 1428.67


In [104]:
dt_grid = {'max_depth': [3, 5, None],
            'max_features': ['sqrt', 'log2', None],
            'min_samples_split': [1, 2, 4],
            'min_samples_leaf': [10, 12, 15, 17, 20],
            'random_state': [1, None]}

dt_model = decision_tree_train_validaion(dt_grid, X_all_train, y_all_train, 10)

Best parameters: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 15, 'min_samples_split': 2, 'random_state': 1}
Decision Tree RMSE: 1440.03


In [111]:
rf_grid = {'max_depth': [3,5, None],
            'max_features': ['sqrt', 'log2', None],
            'min_samples_split': [2, 4, 6],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False],
            'n_estimators': [80, 100, 120],
            'random_state': [1, None]}

rf_model = random_forest_train_validaion(rf_grid, X_all_train, y_all_train, 10)

Best parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 100, 'random_state': None}
Random Forest RMSE: 1096.57


In [119]:
gb_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.5],
              'max_depth': [3,5, None],
              'max_features': ['sqrt', 'log2', None],
              'min_samples_leaf': [ 2, 4,6],
              'n_estimators': [20, 40, 80, 100],
              'random_state': [1, None]}

gb_model = gradient_boosting_train_validaion(gb_grid, 1000, X_all_train, y_all_train, 10)

Best parameters: {'learning_rate': 0.05, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'n_estimators': 80, 'random_state': None}
Random Forest RMSE: 1116.33


In [124]:
y_mean_pred = np.full(y_all_test.shape, np.mean(y_all_train))
baseline_score = round(np.sqrt(mean_squared_error(y_all_test,y_mean_pred)),2)
print(f'Final RMSE for Baseline: {baseline_score}')
lin_test_score = final_testing(X_all_train, X_all_test, y_all_train, y_all_test, lin_reg)
ridge_test_score = final_testing(X_all_train, X_all_test, y_all_train, y_all_test, ridge_reg)
dt_test_score = final_testing(X_all_train, X_all_test, y_all_train, y_all_test, dt_model)
rf_test_score = final_testing(X_all_train, X_all_test, y_all_train, y_all_test, rf_model)
gb_test_score = final_testing(X_all_train, X_all_test, y_all_train, y_all_test, gb_model)

Final RMSE for Baseline: 1785.9
Final RMSE for LinearRegression: 1436.02
Final RMSE for Ridge: 1436.27
Final RMSE for DecisionTreeRegressor: 1381.05
Final RMSE for RandomForestRegressor: 1016.21
Final RMSE for GradientBoostingRegressor: 1016.91


#### Train and Test on Data no range

In [152]:
X_nr = data_norange.drop(columns = 'Rental Price')
y_nr = data_norange['Rental Price']
X_nr_train, X_nr_test, y_nr_train, y_nr_test = train_test_split(X_nr, y_nr)

In [153]:
baseline_train_validate(X_nr_train, y_nr_train, 10)
lin_reg_nr = lin_reg_train_validate(X_nr_train, y_nr_train, 10)
print("")
ridge_reg_nr = ridge_ref_train_validate(50, 250, X_nr_train, y_nr_train, 10)

Baseline RMSE: 2123.27
Linear Regression RMSE: 1698.54

Best alpha is: 211.0
Ridge Regression RMSE: 1675.75


In [156]:
dt_grid = {'max_depth': [3, 5, None],
            'max_features': ['sqrt', 'log2', None],
            'min_samples_split': [1, 2, 4],
            'min_samples_leaf': [10, 12, 15, 17, 20],
            'random_state': [1, None]}

dt_model_nr = decision_tree_train_validaion(dt_grid, X_nr_train, y_nr_train, 10)

Best parameters: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 17, 'min_samples_split': 2, 'random_state': 1}
Decision Tree RMSE: 1654.28


In [159]:
rf_grid = {'max_depth': [3,5, None],
            'max_features': ['sqrt', 'log2', None],
            'min_samples_split': [2, 4, 6],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False],
            'n_estimators': [80, 100, 120],
            'random_state': [1, None]}

rf_model_nr = random_forest_train_validaion(rf_grid, X_nr_train, y_nr_train, 10)

Best parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 80, 'random_state': None}
Random Forest RMSE: 1450.66


In [160]:
gb_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.5],
              'max_depth': [3,5, None],
              'max_features': ['sqrt', 'log2', None],
              'min_samples_leaf': [ 2, 4,6],
              'n_estimators': [20, 40, 80, 100],
              'random_state': [1, None]}

gb_model_nr = gradient_boosting_train_validaion(gb_grid, 1000, X_nr_train, y_nr_train, 10)

Best parameters: {'learning_rate': 0.05, 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 2, 'n_estimators': 80, 'random_state': None}
Random Forest RMSE: 1415.92


In [161]:
y_mean_pred = np.full(y_nr_test.shape, np.mean(y_nr_train))
baseline_score = round(np.sqrt(mean_squared_error(y_nr_test,y_mean_pred)),2)
print(f'Final RMSE for Baseline: {baseline_score}')
lin_test_score = final_testing(X_nr_train, X_nr_test, y_nr_train, y_nr_test, lin_reg_nr)
ridge_test_score = final_testing(X_nr_train, X_nr_test, y_nr_train, y_nr_test, ridge_reg_nr)
dt_test_score = final_testing(X_nr_train, X_nr_test, y_nr_train, y_nr_test, dt_model_nr)
rf_test_score = final_testing(X_nr_train, X_nr_test, y_nr_train, y_nr_test, rf_model_nr)
gb_test_score = final_testing(X_nr_train, X_nr_test, y_nr_train, y_nr_test, gb_model_nr)

Final RMSE for Baseline: 2104.05
Final RMSE for LinearRegression: 1568.73
Final RMSE for Ridge: 1569.71
Final RMSE for DecisionTreeRegressor: 1529.51
Final RMSE for RandomForestRegressor: 1214.71
Final RMSE for GradientBoostingRegressor: 1251.35


## Feature Importance